# O. SVD

This task is classical recommender system. It is well known since Netflix competition in 2006-2009. One
of the competitors Simon Funk has [really nice description](http://sifter.org/simon/journal/20061211.html) of his method that uses SGD to find matrix factorization. It is good, because we don't need to deal with huge sparse matrices.

Another useful thing was [suprise](http://surpriselib.com/) library. It does exactly what is required in our task and 
it has convenient methods to get sample data for testing purposes. We will try to implement our own algorithm
to find matrix factorization and compare the results with those received using this library.

In [1]:
import numpy as np

from surprise import SVD, Dataset, accuracy
from surprise.prediction_algorithms.baseline_only import BaselineOnly
from surprise.model_selection import cross_validate, train_test_split

You can load smaller 100K dataset or larger 1M dataset (it has comparable number of ratings/users/movies as in our task).

**100K** contains 100,000 ratings from 1000 users on 1700 movies.

**1M** contains 1,000,000 ratings from 6000 users on 4000 movies.

In [4]:
# data = Dataset.load_builtin('ml-100k')
data = Dataset.load_builtin('ml-1m')

In [5]:
trainset, testset = train_test_split(data, test_size=.2)

First, let's try baseline algorithm, that doesn't take into account user-movie interaction but just uses global 
average rating, per movie average and per user average rating:
$\hat{r}_{ui} = \mu + b_u + b_i$

In [18]:
model = BaselineOnly()
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.9069


0.90690923722444172

Now, let's try SVD approach that should produce better results.

In [25]:
%%time
model = SVD(n_factors=20, n_epochs=10, lr_all=0.01)
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8712
CPU times: user 12.6 s, sys: 345 ms, total: 13 s
Wall time: 13.3 s


0.87 vs 0.90 which is a nice improvement.

Let's implement our own SGD to find matrix factorization.

In [20]:
n_factors = 20
n_epochs = 10
init_mean = 0
init_std = .1
lr = .01
reg = .02
n_users = trainset.n_users
n_items = trainset.n_items
global_mean = trainset.global_mean

The algorithm is pretty straightforward. The idea is that we have 4 unknowns (2 matrices and 2 vectors):

$b_u$ - per user average rating minus global average, vector of size `n_users`<br>
$b_i$ - per movie(item) average rating minus global average, vector of size `n_items`<br>
$P$ - matrix with rows representing users of size `n_users x n_factors`<br>
$Q$ - matrix with rows representing movies of size `n_items x n_factors`<br>

We initialize them with some random values and then iterate over each known user-movie-raiting tuple and compute 
error. Then we update just a little bit all the weights in matrices to minimize the error.

In [24]:
%%time
bu = np.zeros(n_users, np.double)
bi = np.zeros(n_items, np.double)
pu = np.random.normal(init_mean, init_std, (n_users, n_factors))
qi = np.random.normal(init_mean, init_std, (n_items, n_factors))

for current_epoch in range(n_epochs):
    for u, i, r in trainset.all_ratings():
        err = r - (global_mean + bu[u] + bi[i] + qi[i] @ pu[u])
        
        bu[u] += lr * (err - reg * bu[u])
        bi[i] += lr * (err - reg * bi[i])

        pu[u] += lr * (err * qi[i] - reg * pu[u])
        qi[i] += lr * (err * pu[u] - reg * qi[i])

CPU times: user 1min 38s, sys: 1 s, total: 1min 39s
Wall time: 1min 38s


It takes a bit longer than surprise library. But let's hope the RMSE is in on par.

In [23]:
rmse = 0
predictions = []
for u, i, r in testset:
    est = global_mean
    if u in trainset._raw2inner_id_users:
        est += bu[trainset.to_inner_uid(u)]
    if i in trainset._raw2inner_id_items:
        est += bi[trainset.to_inner_iid(i)]
    if u in trainset._raw2inner_id_users and i in trainset._raw2inner_id_items:
        est += qi[trainset.to_inner_iid(i)] @ pu[trainset.to_inner_uid(u)]
    lower_bound, higher_bound = trainset.rating_scale
    est = min(higher_bound, est)
    est = max(lower_bound, est)
    predictions.append(dict(u=u, i=i, est=est))
    rmse += (r - est) ** 2
rmse = (rmse / len(testset)) ** 0.5
rmse

0.87152340556689434

We've got pretty much the same RMSE 🙌 Small discrepancy might be due random initialization.